In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
# function to read in the data, and set the second row as the column names and drop the first two rows
def read_data(file_name):
    df = pd.read_excel(file_name, sheet_name='Peer View')
    df.columns = df.iloc[1]
    df = df.drop([0,1])
    return df

In [26]:
df_2017 = read_data('Peer View Table for AAPL.O-3.xlsx')
df_2018 = read_data('Peer View Table for AAPL.O-4.xlsx')
df_2019 = read_data('Peer View Table for AAPL.O-5.xlsx')
df_2020 = read_data('Peer View Table for AAPL.O-6.xlsx')
df_2021 = read_data('Peer View Table for AAPL.O-7.xlsx')

In [27]:
df_2017.head()

1,Rank,Company Name,HQ,Industry Group,Mkt. Cap (M),ESG Score
2,811,Apple Inc,US,"Computers, Phones & Household Electronics","$2,336,425",68.27
3,1,BNP Paribas SA,FR,Banking Services,"$65,218",94.26
4,2,AstraZeneca PLC,GB,Pharmaceuticals,"$192,349",93.84
5,3,Stellantis NV,NL,Automobiles & Auto Parts,"$46,668",93.79
6,4,STMicroelectronics NV,CH,Semiconductors & Semiconductor Equipment,"$33,377",93.77


In [35]:
# add a year column to each dataframe
df_2017['Year'] = 2017
df_2018['Year'] = 2018
df_2019['Year'] = 2019
df_2020['Year'] = 2020
df_2021['Year'] = 2021

In [38]:
# merge the dataframes into one
df = pd.concat([df_2017, df_2018, df_2019, df_2020, df_2021], ignore_index=True)
df.tail()

1,Rank,Company Name,HQ,Industry Group,Mkt. Cap (M),ESG Score,Year
39950,8474,Bank of Greece,GR,Banking Services,$325,2.62,2021
39951,8475,Round One Corp,JP,Hotels & Entertainment Services,"$1,258",2.50,2021
39952,8476,MACOM Technology Solutions Holdings Inc,US,Semiconductors & Semiconductor Equipment,"$4,833",2.26,2021
39953,8477,Sandpiper Digital Payments AG,CH,Office Equipment,$2,2.23,2021
39954,8478,Asustek Computer Inc,TW,"Computers, Phones & Household Electronics","$5,896",1.35,2021


In [48]:
# get rid of the dollar sign in Mkt. Cap (M) column
df['Mkt. Cap (M)'] = df['Mkt. Cap (M)'].str.replace('$', '')
# also get rid of the comma in the same column
df['Mkt. Cap (M)'] = df['Mkt. Cap (M)'].str.replace(',', '')
# get the average Market Cap for each company throughout the years, that's getting Mkt. Cap (M) column 
df_avg = df.groupby('Company Name')['Mkt. Cap (M)'].mean().reset_index()



/var/folders/7f/zzvg3mdj425g3y3pzll4q80h0000gn/T/ipykernel_94968/1885840444.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Mkt. Cap (M)'] = df['Mkt. Cap (M)'].str.replace('$', '')


In [50]:
# rank the companies and find the medium
# split the data into two groups, one with the companies that have a market cap above the median, and one with the companies that have a market cap below the median
df_avg['Rank'] = df_avg['Mkt. Cap (M)'].rank(ascending=False)
df_avg_low = df_avg[df_avg['Rank'] > df_avg['Rank'].median()]
df_avg_high = df_avg[df_avg['Rank'] <= df_avg['Rank'].median()]

# count the number of companies in each group
print('Number of companies with a market cap above the median: ', len(df_avg_high))
print('Number of companies with a market cap below the median: ', len(df_avg_low))


Number of companies with a market cap above the median:  5356
Number of companies with a market cap below the median:  5355
